## SNPs matrix

In [1]:
%env DATASET_78644055_VCF_DIR=gs://fc-secure-28df46b0-6f9d-4443-ae5f-cb0492e90c24/genomic-extractions/aee0f862-6c6c-46ea-9f5d-944099fad1be/vcfs

env: DATASET_78644055_VCF_DIR=gs://fc-secure-28df46b0-6f9d-4443-ae5f-cb0492e90c24/genomic-extractions/aee0f862-6c6c-46ea-9f5d-944099fad1be/vcfs


In [2]:
import os
import subprocess

# The extraction workflow outputs a manifest file upon completion.
manifest_file = os.environ['DATASET_78644055_VCF_DIR'] + '/manifest.txt'

assert subprocess.run(['gsutil', '-q', 'stat', manifest_file]).returncode == 0, (
  "!" * 100 + "\n\n" +
  "VCF extraction has not completed.\n" +
  "Please monitor the extraction sidepanel for completion before continuing.\n\n" +
  "!" * 100
)

print("VCF extraction has completed, continuing")


VCF extraction has completed, continuing


In [3]:
# Confirm Spark is installed.
try:
    import pyspark
except ModuleNotFoundError:
    print("!" * 100 + "\n\n"
          "In the Researcher Workbench, Hail can only be used on a Dataproc cluster.\n"
          "Please use the 'Cloud Analysis Environment' side panel to update your runtime compute type.\n\n" +
          "!" * 100)

# Initialize Hail
import hail as hl
import os
from hail.plot import show

hl.init(default_reference='GRCh38')
hl.plot.output_notebook()

Loading BokehJS ...

/opt/conda/lib/python3.10/site-packages/hail/context.py:352: UserWarning:

Using hl.init with a default_reference argument is deprecated. To set a default reference genome after initializing hail, call `hl.default_reference` with an argument to set the default reference genome.

/opt/conda/lib/python3.10/site-packages/hailtop/aiocloud/aiogoogle/user_config.py:43: UserWarning:

Reading spark-defaults.conf to determine GCS requester pays configuration. This is deprecated. Please use `hailctl config set gcs_requester_pays/project` and `hailctl config set gcs_requester_pays/buckets`.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.0
SparkUI available at http://all-of-us-22602-m.us-central1-c.c.terra-vpc-sc-39ac9e8b.internal:41027
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.130.post1-c69cd67afb8b
LOGGING: writing

Loading BokehJS ...

In [4]:
# Create Hail Matrix table
# This can take a few hours for a dataset with hundreds of participants
workspace_bucket = os.environ['WORKSPACE_BUCKET']
vcf_dir = os.environ['DATASET_78644055_VCF_DIR']
hail_matrix_table_gcs = f'{workspace_bucket}/dataset_78644055.mt'
#hl.import_vcf(f'{vcf_dir}/*.vcf.gz', force_bgz=True, array_elements_required=False).write(hail_matrix_table_gcs)


In [5]:
mt = hl.read_matrix_table(hail_matrix_table_gcs)
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_QUALapprox: str, 
        AS_VQSLOD: array<str>, 
        AS_YNG: array<str>, 
        QUALapprox: int32
    }
----------------------------------------
Entry fields:
    'AD': array<int32>
    'FT': str
    'GQ': int32
    'GT': call
    'RGQ': int32
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [22]:
mt.count()

(91932695, 3879)

In [6]:
mt.rows().show()

+---------------+----------------+------+-----------+---------------------+
| locus         | alleles        | rsid |      qual | filters             |
+---------------+----------------+------+-----------+---------------------+
| locus<GRCh38> | array<str>     | str  |   float64 | set<str>            |
+---------------+----------------+------+-----------+---------------------+
| chr1:10108    | ["C","CA"]     | NA   | -1.00e+01 | NA                  |
| chr1:10109    | ["AACCCT","A"] | NA   | -1.00e+01 | {"ExcessHet"}       |
| chr1:10111    | ["C","A"]      | NA   | -1.00e+01 | NA                  |
| chr1:10114    | ["T","A"]      | NA   | -1.00e+01 | NA                  |
| chr1:10119    | ["CT","C"]     | NA   | -1.00e+01 | NA                  |
| chr1:10120    | ["T","A"]      | NA   | -1.00e+01 | NA                  |
| chr1:10122    | ["A","AC"]     | NA   | -1.00e+01 | NA                  |
| chr1:10126    | ["T","A"]      | NA   | -1.00e+01 | NA                  |
| chr1:10128    | ["A","AC"]     | NA   | -1.00e+01 | {"ExcessHet"}       |
| chr1:10132    | ["T","A"]      | NA   | -1.00e+01 | {"NO_HQ_GENOTYPES"} |
+---------------+----------------+------+-----------+---------------------+

+--------------+----------------+---------+--------------------+
| info.AC      | info.AF        | info.AN | info.AS_QUALapprox |
+--------------+----------------+---------+--------------------+
| array<int32> | array<float64> |   int32 | str                |
+--------------+----------------+---------+--------------------+
| [9]          | [4.50e-01]     |      20 | "0|308"            |
| [563]        | [4.71e-01]     |    1196 | "0|16898"          |
| NA           | NA             |      NA | "0|39"             |
| NA           | NA             |      NA | "0|41"             |
| [1]          | [2.62e-04]     |    3820 | "0|6"              |
| NA           | NA             |      NA | "0|1203"           |
| [0]          | [0.00e+00]     |     670 | "0|43"             |
| NA           | NA             |      NA | "0|1055"           |
| [67]         | [4.35e-01]     |     154 | "0|4014"           |
| [8]          | [5.00e-01]     |      16 | "0|294"            |
+--------------+----------------+---------+--------------------+

+----------------+-------------+-----------------+
| info.AS_VQSLOD | info.AS_YNG | info.QUALapprox |
+----------------+-------------+-----------------+
| array<str>     | array<str>  |           int32 |
+----------------+-------------+-----------------+
| ["-2.5895"]    | ["G"]       |              34 |
| NA             | NA          |              33 |
| ["-31.0331"]   | ["G"]       |              39 |
| ["-93.7078"]   | ["G"]       |              41 |
| ["-1.1411"]    | ["G"]       |               6 |
| ["-120.168"]   | ["G"]       |              39 |
| ["-15.7666"]   | ["G"]       |              43 |
| ["-116.9977"]  | ["G"]       |              37 |
| NA             | NA          |              62 |
| NA             | NA          |              37 |
+----------------+-------------+-----------------+
showing top 10 rows

In [7]:
mt = mt.annotate_rows(snp_id=mt.locus.contig.replace("chr", "") + ":" + hl.str(mt.locus.position) + ":" + mt.alleles[0] + ":" + mt.alleles[1])
mt.rows().show()

+---------------+----------------+------+-----------+---------------------+
| locus         | alleles        | rsid |      qual | filters             |
+---------------+----------------+------+-----------+---------------------+
| locus<GRCh38> | array<str>     | str  |   float64 | set<str>            |
+---------------+----------------+------+-----------+---------------------+
| chr1:10108    | ["C","CA"]     | NA   | -1.00e+01 | NA                  |
| chr1:10109    | ["AACCCT","A"] | NA   | -1.00e+01 | {"ExcessHet"}       |
| chr1:10111    | ["C","A"]      | NA   | -1.00e+01 | NA                  |
| chr1:10114    | ["T","A"]      | NA   | -1.00e+01 | NA                  |
| chr1:10119    | ["CT","C"]     | NA   | -1.00e+01 | NA                  |
| chr1:10120    | ["T","A"]      | NA   | -1.00e+01 | NA                  |
| chr1:10122    | ["A","AC"]     | NA   | -1.00e+01 | NA                  |
| chr1:10126    | ["T","A"]      | NA   | -1.00e+01 | NA                  |
| chr1:10128    | ["A","AC"]     | NA   | -1.00e+01 | {"ExcessHet"}       |
| chr1:10132    | ["T","A"]      | NA   | -1.00e+01 | {"NO_HQ_GENOTYPES"} |
+---------------+----------------+------+-----------+---------------------+

+--------------+----------------+---------+--------------------+
| info.AC      | info.AF        | info.AN | info.AS_QUALapprox |
+--------------+----------------+---------+--------------------+
| array<int32> | array<float64> |   int32 | str                |
+--------------+----------------+---------+--------------------+
| [9]          | [4.50e-01]     |      20 | "0|308"            |
| [563]        | [4.71e-01]     |    1196 | "0|16898"          |
| NA           | NA             |      NA | "0|39"             |
| NA           | NA             |      NA | "0|41"             |
| [1]          | [2.62e-04]     |    3820 | "0|6"              |
| NA           | NA             |      NA | "0|1203"           |
| [0]          | [0.00e+00]     |     670 | "0|43"             |
| NA           | NA             |      NA | "0|1055"           |
| [67]         | [4.35e-01]     |     154 | "0|4014"           |
| [8]          | [5.00e-01]     |      16 | "0|294"            |
+--------------+----------------+---------+--------------------+

+----------------+-------------+-----------------+--------------------+
| info.AS_VQSLOD | info.AS_YNG | info.QUALapprox | snp_id             |
+----------------+-------------+-----------------+--------------------+
| array<str>     | array<str>  |           int32 | str                |
+----------------+-------------+-----------------+--------------------+
| ["-2.5895"]    | ["G"]       |              34 | "1:10108:C:CA"     |
| NA             | NA          |              33 | "1:10109:AACCCT:A" |
| ["-31.0331"]   | ["G"]       |              39 | "1:10111:C:A"      |
| ["-93.7078"]   | ["G"]       |              41 | "1:10114:T:A"      |
| ["-1.1411"]    | ["G"]       |               6 | "1:10119:CT:C"     |
| ["-120.168"]   | ["G"]       |              39 | "1:10120:T:A"      |
| ["-15.7666"]   | ["G"]       |              43 | "1:10122:A:AC"     |
| ["-116.9977"]  | ["G"]       |              37 | "1:10126:T:A"      |
| NA             | NA          |              62 | "1:10128:A:AC"     |
| NA             | NA          |              37 | "1:10132:T:A"      |
+----------------+-------------+-----------------+--------------------+
showing top 10 rows

In [6]:
mt = mt.annotate_rows(snp_id=mt.locus.contig.replace("chr", "") + ":" + hl.str(mt.locus.position) + ":" + mt.alleles[0] + ":" + mt.alleles[1])
snp_ids = [
    "14:104920174:G:A", "6:159082054:A:G", "14:68287978:G:A",
    "6:36414159:G:GA", "13:39781776:T:C", "12:45976333:C:G",
    "12:111446804:T:C", "9:34710263:G:A", "5:143224856:A:G",
    "1:116738074:C:T"
]
# Filter the rows to keep only SNPs of interest
snp_set = hl.set(snp_ids)
mt_filtered = mt.filter_rows(snp_set.contains(mt.snp_id))


In [7]:
mt_filtered = mt_filtered.checkpoint("ra_case_filtered_mt_checkpoint.mt", overwrite=True)


2025-02-07 02:49:55.812 Hail: INFO: wrote matrix table with 10 rows and 3879 columns in 2063 partitions to ra_case_filtered_mt_checkpoint.mt


In [10]:
mt_filtered.rows().show()

+-----------------+------------+------+-----------+----------+--------------+
| locus           | alleles    | rsid |      qual | filters  | info.AC      |
+-----------------+------------+------+-----------+----------+--------------+
| locus<GRCh38>   | array<str> | str  |   float64 | set<str> | array<int32> |
+-----------------+------------+------+-----------+----------+--------------+
| chr1:116738074  | ["C","T"]  | NA   | -1.00e+01 | NA       | [4676]       |
| chr5:143224856  | ["A","G"]  | NA   | -1.00e+01 | NA       | [1264]       |
| chr6:36414159   | ["G","GA"] | NA   | -1.00e+01 | NA       | [4407]       |
| chr6:159082054  | ["A","G"]  | NA   | -1.00e+01 | NA       | [1115]       |
| chr9:34710263   | ["G","A"]  | NA   | -1.00e+01 | NA       | [1859]       |
| chr12:45976333  | ["C","G"]  | NA   | -1.00e+01 | NA       | [3252]       |
| chr12:111446804 | ["T","C"]  | NA   | -1.00e+01 | NA       | [4755]       |
| chr13:39781776  | ["T","C"]  | NA   | -1.00e+01 | NA       | [482]        |
| chr14:68287978  | ["G","A"]  | NA   | -1.00e+01 | NA       | [3640]       |
| chr14:104920174 | ["G","A"]  | NA   | -1.00e+01 | NA       | [7313]       |
+-----------------+------------+------+-----------+----------+--------------+

+----------------+---------+--------------------+----------------+-------------+
| info.AF        | info.AN | info.AS_QUALapprox | info.AS_VQSLOD | info.AS_YNG |
+----------------+---------+--------------------+----------------+-------------+
| array<float64> |   int32 | str                | array<str>     | array<str>  |
+----------------+---------+--------------------+----------------+-------------+
| [6.03e-01]     |    7756 | "0|449310"         | ["20.0147"]    | ["Y"]       |
| [1.63e-01]     |    7758 | "0|116057"         | ["20.3526"]    | ["Y"]       |
| [5.68e-01]     |    7758 | "0|423254"         | ["19.5238"]    | ["Y"]       |
| [1.44e-01]     |    7758 | "0|107097"         | ["16.2122"]    | ["Y"]       |
| [2.40e-01]     |    7758 | "0|179047"         | ["20.6603"]    | ["Y"]       |
| [4.19e-01]     |    7758 | "0|324309"         | ["20.4946"]    | ["Y"]       |
| [6.13e-01]     |    7758 | "0|469448"         | ["20.503"]     | ["Y"]       |
| [6.20e-02]     |    7758 | "0|45349"          | ["19.9256"]    | ["Y"]       |
| [4.69e-01]     |    7758 | "0|337063"         | ["20.1817"]    | ["Y"]       |
| [9.43e-01]     |    7758 | "0|769269"         | ["19.5663"]    | ["G"]       |
+----------------+---------+--------------------+----------------+-------------+

+-----------------+--------------------+
| info.QUALapprox | snp_id             |
+-----------------+--------------------+
|           int32 | str                |
+-----------------+--------------------+
|             172 | "1:116738074:C:T"  |
|              85 | "5:143224856:A:G"  |
|             180 | "6:36414159:G:GA"  |
|             178 | "6:159082054:A:G"  |
|              85 | "9:34710263:G:A"   |
|             193 | "12:45976333:C:G"  |
|              85 | "12:111446804:T:C" |
|              85 | "13:39781776:T:C"  |
|             157 | "14:68287978:G:A"  |
|             208 | "14:104920174:G:A" |
+-----------------+--------------------+

In [17]:
# Annotate entries with the genotype allele count (number of alternate alleles per individual)
mt_snps = mt_filtered.annotate_entries(allele_count=hl.case()
                         .when(mt_filtered.GT.is_hom_ref(), 0)  # Homozygous reference → 0 alt alleles
                         .when(mt_filtered.GT.is_het(), 1)      # Heterozygous → 1 alt allele
                         .when(mt_filtered.GT.is_hom_var(), 2)  # Homozygous alternate → 2 alt alleles
                         .or_missing())  # Missing data remains missing


In [18]:
# Extract only the necessary columns
table = mt_snps.entries()
table = table.key_by()
table = table.select('s', 'snp_id', 'allele_count')

2025-02-07 03:18:42.170 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'


In [56]:
# Convert to a wide format: row = individuals, columns = SNPs
snp_matrix = table.to_pandas().pivot(index="s", columns="snp_id", values="allele_count") 

In [67]:
snp_matrix = snp_matrix.reset_index().rename(columns={"s": "person_id"})
snp_matrix['person_id'] = snp_matrix['person_id'].astype(int)  # Convert to integer

In [68]:
snp_matrix.shape

(3879, 11)

In [69]:
snp_matrix.iloc[0:10,]

snp_id,person_id,12:111446804:T:C,12:45976333:C:G,13:39781776:T:C,14:104920174:G:A,14:68287978:G:A,1:116738074:C:T,5:143224856:A:G,6:159082054:A:G,6:36414159:G:GA,9:34710263:G:A
0,1000042,2,2,0,1,2,1,0,2,2,2
1,1000059,1,2,1,2,2,0,2,1,1,1
2,1003242,2,2,1,2,0,2,1,1,1,2
3,1004690,1,2,1,1,0,1,1,2,1,1
4,1004776,0,1,1,2,2,2,1,1,2,1
5,1005022,0,2,2,2,2,1,0,0,2,1
6,1006298,0,2,2,2,2,1,0,2,2,0
7,1006557,2,0,0,2,0,1,1,1,2,2
8,1006836,1,2,1,2,1,2,1,2,2,2
9,1007317,2,1,2,2,2,2,1,1,1,1


In [70]:
snp_matrix.isna().sum()

snp_id
person_id           0
12:111446804:T:C    0
12:45976333:C:G     0
13:39781776:T:C     0
14:104920174:G:A    0
14:68287978:G:A     0
1:116738074:C:T     1
5:143224856:A:G     0
6:159082054:A:G     0
6:36414159:G:GA     0
9:34710263:G:A      0
dtype: int64

In [71]:
snp_matrix.to_csv("ra_case_selected_snp_matrix.csv", index=False)